# Apify Actor

>[Apify Actors](https://docs.apify.com/platform/actors) are cloud programs designed for a wide range of web scraping, crawling, and data extraction tasks. These actors facilitate automated data gathering from the web, enabling users to extract, process, and store information efficiently. Actors can be used to perform tasks like scraping e-commerce sites for product details, monitoring price changes, or gathering search engine results. They integrate seamlessly with [Apify Datasets](https://docs.apify.com/platform/storage/dataset), allowing the structured data collected by actors to be stored, managed, and exported in formats like JSON, CSV, or Excel for further analysis or use.

This notebook shows how to use Apify Actors with LangChain.


## Prerequisites

- **Apify account**: Register your free Apify account [here](https://console.apify.com/sign-up).
- **Apify API token**: Learn how to get your API token in the [Apify documentation](https://docs.apify.com/platform/integrations/api).

In [ ]:
%pip install langgraph langchain-apify langchain-openai

First, import `ApifyActorsTool` into your source code:

In [2]:
from langchain_apify import ApifyActorsTool

Find your [Apify API token](https://console.apify.com/account/integrations) and [OpenAI API key](https://platform.openai.com/account/api-keys) and initialize these into environment variables:

In [3]:
import os

os.environ["APIFY_API_TOKEN"] = "your-apify-api-token"
os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

# Using Apify Actors as LangChain agent tools

We will create an `ApifyActorsTool` instance to be able to call Apify Actors and pass it to an agent. Any Actors from the [Apify Store](https://apify.com/store) can be used as a tool. In this example, we will use the [Website Content Crawler](https://apify.com/apify/website-content-crawler) to extract and analyze website content.

In [4]:
wcc = ApifyActorsTool("apify/website-content-crawler")
tools = [wcc]

## Providing Apify Actors tool to an agent

We can provide the created tool to an agent. When asked to summarize a website, the agent will call the Apify Actor to extract the content of the website and then return a summary of the retrieved content.


In [10]:
from langchain_core.messages import ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

model = ChatOpenAI(model="gpt-4o")
graph = create_react_agent(model, tools=tools)

Run the agent to summarize the content of [whitepaper.actor](https://whitepaper.actor/).

In [12]:
inputs = {
    "messages": [("user", "get and summarize me content of this site whitepaper.actor")]
}
for s in graph.stream(inputs, stream_mode="values"):
    message = s["messages"][-1]
    # skip tool messages
    if isinstance(message, ToolMessage):
        continue
    message.pretty_print()

================================ Human Message =================================

get and summarize me content of this site whitepaper.actor
================================== Ai Message ==================================
Tool Calls:
  apify_actor_apify_website-content-crawler (call_VkGocyUlaMUqhm8fHJhi5Z7p)
 Call ID: call_VkGocyUlaMUqhm8fHJhi5Z7p
  Args:
    run_input: {"startUrls":[{"url":"https://whitepaper.actor"}],"proxyConfiguration":{"useApifyProxy":true},"maxCrawlPages":5,"maxResults":3}
================================== Ai Message ==================================

The **Web Actor Programming Model Whitepaper** introduces a new way to build serverless microapps called Actors. These are cloud-based tools that allow developers to create and distribute software effortlessly. Here are the key takeaways:

1. **Overview of Actors:**
   - Actors are serverless programs that perform various tasks from simple actions like sending an email to complex ones such as web crawling.
   - Th